# Preprocessamento

Este notebook é para tirar os dados do `.csv` e migrar eles para o Postgres.

## 1. Preparando o ambiente

Antes de iniciarmos a execução do projeto, é fundamental preparar o ambiente Python. Este repositório utiliza Pipenv como gerenciador de dependências e ambientes virtuais. Portanto, recomenda-se que você tenha o Pipenv instalado em sua máquina local.

Após confirmar a instalação, execute o seguinte comando na raiz do projeto:

```bash
pipenv install
```

Esse comando irá instalar automaticamente todas as dependências necessárias para a reprodução e execução do projeto.

In [1]:
import csv
import sys
import numpy
import pandas
import sqlalchemy
import sqlalchemy.orm as orm
import os
import json
import re
import html
import unicodedata
import ast
from dotenv import load_dotenv
from datetime import datetime

In [2]:
# tamanho do batch para inserção no banco
BATCH_SIZE = 2_500

# caminho do arquivo corrigido
CSV_FIXED_PATH = "./datasets/games_fixed.csv"

In [3]:
# remove o limite de tamanho de campo do csv
csv.field_size_limit(sys.maxsize)

# mostra todas as colunas do dataframe
pandas.set_option('display.max_columns', None)

## 2. Preparando o PostgreSQL

Antes de migrarmos para o PostgreSQL, é necessário preparar o banco de dados e suas tabelas para receber os dados do projeto.

Recomendamos utilizar o Docker, pois isso simplifica a criação e execução de um contêiner com PostgreSQL na máquina local. Caso ainda não tenha o Docker instalado, consulte as instruções oficiais disponíveis em: https://docs.docker.com/engine/install/

Este repositório inclui um arquivo compose.yml e um arquivo de exemplo .env.example. Para configurar o ambiente:

1. Crie um arquivo .env na raiz do projeto.

2. Copie o conteúdo de .env.example para dentro do novo arquivo .env.

Com isso feito, execute o comando abaixo para inicializar um contêiner com PostgreSQL:

```bash
docker compose up -d
```

Esse comando iniciará o banco de dados em segundo plano, permitindo que você continue utilizando o terminal enquanto o serviço é executado.

### 2.1 Conectando com o Banco de Dados PostgreSQL

Após configurar o ambiente e iniciar o servidor PostgreSQL via Docker, o próximo passo é estabelecer a conexão entre o projeto Python e o banco de dados. Para isso, utilizamos o pacote SQLAlchemy, que fornece uma interface de alto nível para comunicação com bancos relacionais, e o pacote python-dotenv, responsável por carregar as variáveis de ambiente definidas no arquivo `.env`.

#### 2.1.1. Carregando variáveis de ambiente

As credenciais de acesso ao banco (host, porta, usuário, senha e nome do banco) são armazenadas no arquivo `.env`. Isso evita que informações sensíveis fiquem expostas diretamente no código. No início do script, utilizamos:

In [4]:
load_dotenv()

True

Esse comando carrega todas as variáveis definidas no `.env` para o ambiente de execução, permitindo acessá-las via `os.getenv()`.

#### 2.1.2. Validando as configurações

Para garantir que todas as variáveis necessárias foram informadas, o código realiza uma verificação simples. Caso alguma esteja ausente, uma exceção é lançada. Essa etapa evita falhas silenciosas e facilita o diagnóstico de erros de configuração.

In [5]:
# Carregar variáveis de ambiente
POSTGRES_HOST = os.getenv("POSTGRES_HOST")
POSTGRES_PORT = os.getenv("POSTGRES_PORT")
POSTGRES_USER = os.getenv("POSTGRES_USER")
POSTGRES_PASSWORD = os.getenv("POSTGRES_PASSWORD")
POSTGRES_DB = os.getenv("POSTGRES_DB")

# Verificar se todas as variáveis foram carregadas
required_vars = {
    "POSTGRES_HOST": POSTGRES_HOST,
    "POSTGRES_PORT": POSTGRES_PORT,
    "POSTGRES_USER": POSTGRES_USER,
    "POSTGRES_PASSWORD": POSTGRES_PASSWORD,
    "POSTGRES_DB": POSTGRES_DB,
}

missing = [k for k, v in required_vars.items() if not v]
if missing:
    raise EnvironmentError(f"Variáveis de ambiente ausentes: {', '.join(missing)}")

#### 2.1.3. Construindo a URL de conexão

Com as variáveis carregadas, construímos a URL de conexão no formato compatível com o driver `psycopg` do SQLAlchemy:

In [6]:
DATABASE_URL = (
    f"postgresql+psycopg://{POSTGRES_USER}:{POSTGRES_PASSWORD}"
    f"@{POSTGRES_HOST}:{POSTGRES_PORT}/{POSTGRES_DB}"
)

Essa URL contém todas as informações necessárias para que o SQLAlchemy saiba como se conectar ao banco.

#### 2.1.4. Criando a engine do SQLAlchemy

A engine é o objeto central responsável por gerenciar conexões e enviar comandos ao banco:

In [7]:
engine = sqlalchemy.create_engine(DATABASE_URL)

Ela serve como ponto de entrada para operações como consultas, inserções e criação de tabelas.

#### 2.1.5. Testando a conexão

Por fim, o código tenta abrir uma conexão simples com o banco. Caso seja bem-sucedido, uma mensagem de sucesso é exibida; caso contrário, a exceção é capturada e exibida:

In [8]:
try:
    with engine.connect() as connection:
        print("✅ Conexão com o banco de dados estabelecida com sucesso!")
except Exception as e:
    print(f"❌ Erro ao conectar ao banco de dados: {e}")

✅ Conexão com o banco de dados estabelecida com sucesso!


Essa verificação é importante para garantir que o restante do notebook possa interagir com o banco normalmente.

### 2.2 Modelagem do Banco de Dados com SQLAlchemy

A seguir, apresentamos a definição das tabelas e classes que compõem o modelo relacional utilizado neste projeto. A modelagem foi implementada com SQLAlchemy ORM, que permite mapear tabelas e relacionamentos do banco de dados para classes Python, facilitando operações de consulta, inserção e manipulação de dados.

#### 2.2.1. Base declarativa

In [9]:
Base = orm.declarative_base()

O SQLAlchemy utiliza uma base declarativa como ponto central para registrar todas as classes (models) e metadados das tabelas. Todas as entidades do banco herdam dessa `Base`, o que permite ao framework gerar automaticamente as tabelas no PostgreSQL.

#### 2.2.2. Tabelas de associação (relacionamentos M:N)

Muitos dos relacionamentos entre jogos (games) e outros elementos (como desenvolvedores, categorias, gêneros, tags e idiomas) são do tipo muitos-para-muitos (M:N). Em um modelo relacional tradicional, esse tipo de relação deve ser representado por tabelas pivô (também chamadas de tabelas de associação ou junction tables).

Por exemplo, um jogo pode ter vários desenvolvedores, e um desenvolvedor pode estar associado a vários jogos. Esse padrão se repete para publishers, categorias, gêneros, tags e idiomas.

Cada tabela de associação é criada da seguinte forma:

In [10]:
game_developer = sqlalchemy.Table(
    "game_developer",
    Base.metadata,
    sqlalchemy.Column("game_id", sqlalchemy.Integer, sqlalchemy.ForeignKey("games.id"), primary_key=True),
    sqlalchemy.Column("developer_id", sqlalchemy.Integer, sqlalchemy.ForeignKey("developers.id"), primary_key=True)
)

game_publisher = sqlalchemy.Table(
    "game_publisher",
    Base.metadata,
    sqlalchemy.Column("game_id", sqlalchemy.Integer, sqlalchemy.ForeignKey("games.id"), primary_key=True),
    sqlalchemy.Column("publisher_id", sqlalchemy.Integer, sqlalchemy.ForeignKey("publishers.id"), primary_key=True)
)

game_category = sqlalchemy.Table(
    "game_category",
    Base.metadata,
    sqlalchemy.Column("game_id", sqlalchemy.Integer, sqlalchemy.ForeignKey("games.id"), primary_key=True),
    sqlalchemy.Column("category_id", sqlalchemy.Integer, sqlalchemy.ForeignKey("categories.id"), primary_key=True)
)

game_genre = sqlalchemy.Table(
    "game_genre",
    Base.metadata,
    sqlalchemy.Column("game_id", sqlalchemy.Integer, sqlalchemy.ForeignKey("games.id"), primary_key=True),
    sqlalchemy.Column("genre_id", sqlalchemy.Integer, sqlalchemy.ForeignKey("genres.id"), primary_key=True)
)

game_tag = sqlalchemy.Table(
    "game_tag",
    Base.metadata,
    sqlalchemy.Column("game_id", sqlalchemy.Integer, sqlalchemy.ForeignKey("games.id"), primary_key=True),
    sqlalchemy.Column("tag_id", sqlalchemy.Integer, sqlalchemy.ForeignKey("tags.id"), primary_key=True)
)

game_language = sqlalchemy.Table(
    "game_language",
    Base.metadata,
    sqlalchemy.Column("game_id", sqlalchemy.Integer, sqlalchemy.ForeignKey("games.id"), primary_key=True),
    sqlalchemy.Column("language_id", sqlalchemy.Integer, sqlalchemy.ForeignKey("languages.id"), primary_key=True)
)

#### 2.2.3. Características importantes das tabelas pivô:

- Não possuem classe ORM própria — pois servem apenas como ligação entre duas entidades.

- Cada linha representa um vínculo M:N entre duas tabelas.

- Usam chaves estrangeiras para garantir integridade referencial.

- A combinação de colunas funciona como chave primária composta.

São definidas tabelas equivalentes para publishers, categorias, gêneros, tags e idiomas.

#### 2.2.4. Definição dos modelos (tabelas principais)

Cada entidade principal do banco de dados (Developer, Publisher, Category, Genre, Tag, Language, Game, entre outras) é representada por uma classe que herda da `Base`.

In [11]:
class Developer(Base):
    __tablename__ = "developers"

    id = sqlalchemy.Column(sqlalchemy.Integer, primary_key=True)
    name = sqlalchemy.Column(sqlalchemy.Text, nullable=False)

    games = orm.relationship("Game", secondary=game_developer, back_populates="developers")

    @classmethod
    def get_or_create(cls, name, session):
        parsed = name.strip().lower()
        obj = session.query(cls).filter_by(name=parsed).first()
        if obj:
            return obj
        obj = cls(name=parsed)
        session.add(obj)
        session.commit()
        return obj


class Publisher(Base):
    __tablename__ = "publishers"

    id = sqlalchemy.Column(sqlalchemy.Integer, primary_key=True)
    name = sqlalchemy.Column(sqlalchemy.Text, nullable=False)

    games = orm.relationship("Game", secondary=game_publisher, back_populates="publishers")

    @classmethod
    def get_or_create(cls, name, session):
        parsed = name.strip().lower()
        obj = session.query(cls).filter_by(name=parsed).first()
        if obj:
            return obj
        obj = cls(name=parsed)
        session.add(obj)
        session.commit()
        return obj


class Category(Base):
    __tablename__ = "categories"

    id = sqlalchemy.Column(sqlalchemy.Integer, primary_key=True)
    name = sqlalchemy.Column(sqlalchemy.String(255), nullable=False)

    games = orm.relationship("Game", secondary=game_category, back_populates="categories")

    @classmethod
    def get_or_create(cls, name, session):
        parsed = name.strip().lower()
        obj = session.query(cls).filter_by(name=parsed).first()
        if obj:
            return obj
        obj = cls(name=parsed)
        session.add(obj)
        session.commit()
        return obj


class Genre(Base):
    __tablename__ = "genres"

    id = sqlalchemy.Column(sqlalchemy.Integer, primary_key=True)
    name = sqlalchemy.Column(sqlalchemy.String(255), nullable=False)

    games = orm.relationship("Game", secondary=game_genre, back_populates="genres")

    @classmethod
    def get_or_create(cls, name, session):
        parsed = name.strip().lower()
        obj = session.query(cls).filter_by(name=parsed).first()
        if obj:
            return obj
        obj = cls(name=parsed)
        session.add(obj)
        session.commit()
        return obj


class Tag(Base):
    __tablename__ = "tags"

    id = sqlalchemy.Column(sqlalchemy.Integer, primary_key=True)
    name = sqlalchemy.Column(sqlalchemy.String(255), nullable=False)

    games = orm.relationship("Game", secondary=game_tag, back_populates="tags")

    @classmethod
    def get_or_create(cls, name, session):
        parsed = name.strip().lower()
        obj = session.query(cls).filter_by(name=parsed).first()
        if obj:
            return obj
        obj = cls(name=parsed)
        session.add(obj)
        session.commit()
        return obj


class Language(Base):
    __tablename__ = "languages"

    id = sqlalchemy.Column(sqlalchemy.Integer, primary_key=True)
    name = sqlalchemy.Column(sqlalchemy.String(255), nullable=False)

    games = orm.relationship("Game", secondary=game_language, back_populates="languages")

    @classmethod
    def get_or_create(cls, name, session):
        parsed = name.strip().lower()
        obj = session.query(cls).filter_by(name=parsed).first()
        if obj:
            return obj
        obj = cls(name=parsed)
        session.add(obj)
        session.commit()
        return obj


class Screenshot(Base):
    __tablename__ = "screenshots"

    id = sqlalchemy.Column(sqlalchemy.Integer, primary_key=True)
    game_id = sqlalchemy.Column(sqlalchemy.Integer, sqlalchemy.ForeignKey("games.id"), nullable=False)
    screenshot_url = sqlalchemy.Column(sqlalchemy.String(255), nullable=False)


class Movie(Base):
    __tablename__ = "movies"

    id = sqlalchemy.Column(sqlalchemy.Integer, primary_key=True)
    game_id = sqlalchemy.Column(sqlalchemy.Integer, sqlalchemy.ForeignKey("games.id"), nullable=False)
    movie_url = sqlalchemy.Column(sqlalchemy.String(255), nullable=False)


class Game(Base):
    __tablename__ = "games"

    id = sqlalchemy.Column(sqlalchemy.Integer, primary_key=True)
    app_id = sqlalchemy.Column(sqlalchemy.Integer, nullable=False)
    name = sqlalchemy.Column(sqlalchemy.Text, nullable=False)
    release_date = sqlalchemy.Column(sqlalchemy.Date, nullable=False)
    estimated_owners_lower = sqlalchemy.Column(sqlalchemy.Integer, nullable=False)
    estimated_owners_upper = sqlalchemy.Column(sqlalchemy.Integer, nullable=False)
    peak_ccu = sqlalchemy.Column(sqlalchemy.Integer, nullable=False, default=0)
    required_age = sqlalchemy.Column(sqlalchemy.Integer, nullable=False, default=0)
    price = sqlalchemy.Column(sqlalchemy.Float, nullable=False, default=0.0)
    discount = sqlalchemy.Column(sqlalchemy.Float, nullable=False, default=0.0)
    dlc_count = sqlalchemy.Column(sqlalchemy.Integer, nullable=False, default=0)
    about_the_game = sqlalchemy.Column(sqlalchemy.Text, nullable=True)
    header_image = sqlalchemy.Column(sqlalchemy.Text, nullable=True)
    website = sqlalchemy.Column(sqlalchemy.Text, nullable=True)
    support_url = sqlalchemy.Column(sqlalchemy.Text, nullable=True)
    support_email = sqlalchemy.Column(sqlalchemy.Text, nullable=True)
    windows = sqlalchemy.Column(sqlalchemy.Boolean, nullable=False, default=False)
    mac = sqlalchemy.Column(sqlalchemy.Boolean, nullable=False, default=False)
    linux = sqlalchemy.Column(sqlalchemy.Boolean, nullable=False, default=False)
    metacritic_score = sqlalchemy.Column(sqlalchemy.Float, nullable=True)
    metacritic_url = sqlalchemy.Column(sqlalchemy.Text, nullable=True)
    user_score = sqlalchemy.Column(sqlalchemy.Float, nullable=True)
    positive = sqlalchemy.Column(sqlalchemy.Integer, nullable=True)
    negative = sqlalchemy.Column(sqlalchemy.Integer, nullable=True)
    score_rank = sqlalchemy.Column(sqlalchemy.Integer, nullable=True)
    achievements = sqlalchemy.Column(sqlalchemy.Integer, nullable=True)
    recommendations = sqlalchemy.Column(sqlalchemy.Integer, nullable=True)
    average_playtime_forever = sqlalchemy.Column(sqlalchemy.Integer, nullable=True)
    average_playtime_2weeks = sqlalchemy.Column(sqlalchemy.Integer, nullable=True)
    median_playtime_forever = sqlalchemy.Column(sqlalchemy.Integer, nullable=True)
    median_playtime_2weeks = sqlalchemy.Column(sqlalchemy.Integer, nullable=True)

    # RELACIONAMENTOS M:N
    developers = orm.relationship("Developer", secondary=game_developer, back_populates="games")
    publishers = orm.relationship("Publisher", secondary=game_publisher, back_populates="games")
    categories = orm.relationship("Category", secondary=game_category, back_populates="games")
    genres = orm.relationship("Genre", secondary=game_genre, back_populates="games")
    tags = orm.relationship("Tag", secondary=game_tag, back_populates="games")
    languages = orm.relationship("Language", secondary=game_language, back_populates="games")


#### 2.2.5. Método `get_or_create`

As classes também implementam um método utilitário:

```python
@classmethod
def get_or_create(cls, name, session):
```

Esse método:

- Padroniza o nome (ex.: minúsculas).

- Verifica se o registro já existe.

- Caso não exista, cria um novo.

- Garante consistência dos dados ao evitar duplicações.

### 2.3. Criando as Tabelas no Banco de Dados e Inicializando a Sessão

Após definir todos os modelos e tabelas com o SQLAlchemy, o próximo passo é criar a estrutura física dessas tabelas dentro do banco PostgreSQL e preparar uma sessão para realizar operações de leitura e escrita.

#### 2.3.1 Criando as tabelas no banco

In [12]:
Base.metadata.create_all(engine)

O SQLAlchemy mantém, por meio de Base.metadata, um registro completo de todas as classes (models) e tabelas declaradas no projeto. Ao chamar create_all(engine), o framework:

- Verifica quais tabelas já existem no banco.

- Cria automaticamente apenas as tabelas que ainda não foram criadas.

- Garante a criação das tabelas pivô, modelos principais e relacionamentos.

Esse comando funciona como uma migração inicial simplificada, útil especialmente em desenvolvimento ou quando o esquema do banco ainda não está sendo gerenciado por uma ferramenta de migração mais avançada.

#### 2.3.2 Criando a sessão de comunicação com o banco

In [13]:
session = sqlalchemy.orm.Session(engine)

A sessão (Session) é o componente central do SQLAlchemy ORM para interação com o banco. Ela funciona como:

- Uma unidade de trabalho (unit of work), controlando transações.

- Um gerenciador de objetos, mantendo instâncias do banco sincronizadas com o Python.

- A interface por onde realizamos:

  - consultas (session.query(...))

  - inserções (session.add(...))

  - atualizações

  - deleções

  - commits e rollbacks

## 3. Migrar dados

Para migrarmos os dados, antes temos que carregar eles com o Pandas.

In [14]:
games_dataset = pandas.read_csv(
  CSV_FIXED_PATH,
  sep=",",
  quotechar='"',
  quoting=csv.QUOTE_MINIMAL,
  engine="python",
  encoding="utf-8-sig",
)

### 3.1. Tags

Para migrar as tags

In [15]:
# pega todas as tags da coluna, remove NaN e garante strings
tag_column = games_dataset['Tags'].dropna().astype(str)

unique_tags = set()

# extrai tags únicas
for tag_string in tag_column:
    names = (name.strip().lower() for name in tag_string.split(','))
    unique_tags.update(names)

print("Total únicos encontrados:", len(unique_tags))


# pega do banco as tags já existentes
existing_tags = {
    t.name.lower()
    for t in session.query(Tag).all()
}

# mantém só as realmente novas
new_tags = [
    Tag(name=tag)
    for tag in unique_tags
    if tag not in existing_tags
]

# insere tudo de uma vez (muito mais rápido)
session.bulk_save_objects(new_tags)
session.commit()

print(f"Adicionadas: {len(new_tags)} tags")


Total únicos encontrados: 453
Adicionadas: 453 tags


In [ ]:
# cria um dicionário {nome_lowercase: id} para lookup rápido
tag_map = {
    tag.name.lower(): tag.id
    for tag in session.query(Tag).all()
}

def get_tag_id(tag_names):
    if not isinstance(tag_names, str):
        return None
    
    ids = []
    for name in map(str.strip, tag_names.lower().split(',')):
        tag_id = tag_map.get(name)
        if tag_id is None:
            print(f"Tag {name} não encontrada")
        else:
            ids.append(tag_id)
    return ids

# aplica ao dataset
games_dataset['Tags'] = games_dataset['Tags'].apply(get_tag_id)

print("Tags trocadas com sucesso!")

### 3.2. Desenvolvedores

Para migrar os desenvolvedores

### 3.3. Publicadores

### 3.4. Categorias

### 3.5. Generos

### 3.6. Linguagens

### 3.7. Games